In [51]:
from alpaca_trade_api.rest import REST, TimeFrame
import pandas as pd
from dotenv import dotenv_values

In [52]:
BASE_URL = "https://paper-api.alpaca.markets"

config = dotenv_values(".env")
KEY_ID = config['KEY_ID']
SECRET_KEY = config['SECRET_KEY']

In [53]:
# Instantiate REST API Connection
api = REST(key_id=KEY_ID,secret_key=SECRET_KEY,base_url="https://paper-api.alpaca.markets")

# Fetch 1Minute historical bars of Bitcoin
bars = api.get_crypto_bars("BTC/USD", TimeFrame.Minute).df
bars.index = bars.index.tz_convert('America/Los_Angeles')
bars

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2024-02-27 22:00:00-08:00,57142.8690,57142.8690,57116.6395,0,57116.6395,0.000000,0.0000,BTC/USD
2024-02-27 22:01:00-08:00,57154.9500,57154.9500,57154.9500,0,57154.9500,0.000000,0.0000,BTC/USD
2024-02-27 22:04:00-08:00,57192.5850,57192.5850,57192.5850,0,57192.5850,0.000000,0.0000,BTC/USD
2024-02-27 22:07:00-08:00,57155.1085,57155.1085,57155.1085,0,57155.1085,0.000000,0.0000,BTC/USD
2024-02-27 22:10:00-08:00,57154.5550,57165.7550,57154.5550,0,57165.7550,0.000000,0.0000,BTC/USD
...,...,...,...,...,...,...,...,...
2024-02-28 08:47:00-08:00,61858.4000,61858.4000,61858.4000,1,61858.4000,0.000100,61858.4000,BTC/USD
2024-02-28 08:48:00-08:00,61787.3800,61787.3800,61787.3800,1,61787.3800,0.000019,61787.3800,BTC/USD
2024-02-28 08:49:00-08:00,61806.1000,61806.1000,61806.1000,1,61806.1000,0.010000,61806.1000,BTC/USD


In [54]:
from datetime import datetime, timedelta
import math
import time
import string

SYMBOL = 'BTC/USD'
SMA_FAST = 6
SMA_SLOW = 40
QTY_PER_TRADE = 1.6


def remove_punctuation(text):
    return ''.join(char for char in text if char not in string.punctuation)

# Description is given in the article
def get_pause():
    now = datetime.now()
    next_min = now.replace(second=0, microsecond=0) + timedelta(minutes=1, seconds=1)
    pause = math.ceil((next_min - now).seconds)
    print(f"Sleep for {pause}")
    return pause

# Same as the function in the random version
def get_position(symbol):
    symbol = remove_punctuation(symbol)
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0


# Returns a series with the moving average
def get_sma(series, periods):
    return series.rolling(periods).mean()

# Checks wether we should buy (fast ma > slow ma)
def get_signal(fast, slow):
    print(f"Fast: {fast.iloc[-1]} \t Slow: {slow.iloc[-1]}")
    return fast.iloc[-1] > slow.iloc[-1]

# Get up-to-date 1 minute data from Alpaca and add the moving averages
def get_bars(symbol):
    bars = api.get_crypto_bars(symbol, TimeFrame.Minute).df
    bars[f'sma_fast'] = get_sma(bars.close, SMA_FAST)
    bars[f'sma_slow'] = get_sma(bars.close, SMA_SLOW)
    return bars

while True:
    # GET DATA
    bars = get_bars(symbol=SYMBOL)
    # CHECK POSITIONS
    position = get_position(symbol=SYMBOL)
    should_buy = get_signal(bars.sma_fast,bars.sma_slow)
    if position == 0 and should_buy == True:
        # WE BUY ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='buy', time_in_force='gtc')
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: {QTY_PER_TRADE}')
    elif position > 0 and should_buy == False:
        # WE SELL ONE BITCOIN
        api.submit_order(SYMBOL, qty=position, side='sell', time_in_force='gtc')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: {position}')

    time.sleep(get_pause())
    print('\n\n')

Fast: 61843.54783333332 	 Slow: 61521.829237499995
Sleep for 13



Fast: 61884.59533333333 	 Slow: 61547.68861250001
Sleep for 60



Fast: 61943.682 	 Slow: 61574.0056125
Sleep for 60



Fast: 62019.83833333334 	 Slow: 61601.3029875
Sleep for 59



Fast: 62135.79283333333 	 Slow: 61634.942650000005
Sleep for 60



Fast: 62232.51866666666 	 Slow: 61668.579775000006
Sleep for 60



Fast: 62303.715083333336 	 Slow: 61702.710025
Sleep for 60



Fast: 62333.67833333334 	 Slow: 61732.16309999999
Sleep for 59



Fast: 62356.57 	 Slow: 61759.2108625
Sleep for 60



Fast: 62367.03366666666 	 Slow: 61784.62136250001
Sleep for 60
